In [0]:
display(dbutils.secrets.listScopes())
display(dbutils.secrets.list("practice"))
service_credential = dbutils.secrets.get("practice", "adls-key")
print(service_credential)

application_id = "bada1f1e-8aaa-4bb1-8d1b-72b6101283f6"
directory_id = "4249dcf4-f4a1-44f9-940d-14b50a777dd8"

spark.conf.set("fs.azure.account.auth.type.adls12s.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.adls12s.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.adls12s.dfs.core.windows.net", application_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.adls12s.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.adls12s.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

name
databricks-scope
practice


key
adls-key
databricks-key


[REDACTED]


In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": application_id,
          "fs.azure.account.oauth2.client.secret": service_credential,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{directory_id}/oauth2/token"}

def checkIfMounted(mounts):
    if(any(mount.mountPoint == "/mnt/adls12s/raw/" for mount in mounts )):
        print("Mount point exists")
        dbutils.fs.unmount("/mnt/adls12s/raw/")
        print("Unmounted Successfully")
        dbutils.fs.mount(source = "abfss://raw@adls12s.dfs.core.windows.net/", mount_point = "/mnt/adls12s/raw/", extra_configs = configs)
        print("Mounted Successfully")
    else:
        print("Mount point does not exist")
        dbutils.fs.mount(source = "abfss://raw@adls12s.dfs.core.windows.net/", mount_point = "/mnt/adls12s/raw/", extra_configs = configs)
        print("Mounted Successfully")

mountsList = dbutils.fs.mounts()
display(mountsList)

try:
    dbutils.fs.mount(source = "abfss://raw@adls12s.dfs.core.windows.net/", mount_point = "/mnt/adls12s/raw/", extra_configs = configs)
    print('No existing mount point found, Mounting')
except Exception as e:
    checkIfMounted(mountsList)

mountPoint,source,encryptionType
/mnt/adls12s/gold/,abfss://gold@adls12s.dfs.core.windows.net/,
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/mnt/adls12s/source/,abfss://source@adls12s.dfs.core.windows.net/,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/mnt/adls2sc/raw/,abfss://raw@adls2sc.dfs.core.windows.net/,
/volumes,DbfsReserved,


Mount point exists
/mnt/adls12s/raw/ has been unmounted.
Unmounted Successfully
Mounted Successfully


In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType, TimestampType, FloatType
schema = StructType([StructField("id", IntegerType(), True),
                     StructField("name", StringType(), True),
                     StructField("host_id", IntegerType(), True),
                     StructField("host_name", StringType(), True),
                     StructField("neighbourhood_group", StringType(), True),
                     StructField("neighbourhood", StringType(), True),
                     StructField("latitude", FloatType(), True),
                     StructField("longitude", FloatType(), True),
                     StructField("room_type", StringType(), True),
                     StructField("price", IntegerType(), True),
                     StructField("minimum_nights", IntegerType(), True),
                     StructField("number_of_reviews", IntegerType(), True),
                     StructField("last_review", DateType(), True),
                     StructField("reviews_per_month", FloatType(), True),
                     StructField("calculated_host_listings_count", IntegerType(), True),
                     StructField("availability_365", IntegerType(), True)])

#Load the unclean airbnb data set
df = spark.read.csv(f"/mnt/adls12s/raw/{dbutils.fs.ls('/mnt/adls12s/raw/')[-1].name}", schema=schema, header = True)

#Check Statistics of the data set
display(df.head(5))
display(df.summary())
df.toPandas().info()


#Checking number of Null records in each column
df.toPandas().isnull().sum()

#Check the shape of the dataset
df.toPandas().shape


id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.647491455078125,-73.97236633300781,Private room,149,1,9,2018-10-19,0.20999999344348907,6,365
2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362014770508,-73.98377227783203,Entire home/apt,225,1,45,2019-05-21,0.3799999952316284,2,355
3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902099609375,-73.94190216064453,Private room,150,3,0,null,null,1,365
3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68513870239258,-73.95976257324219,Entire home/apt,89,1,270,2019-07-05,4.639999866485596,1,194
5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851150512695,-73.9439926147461,Entire home/apt,80,10,9,2018-11-19,0.10000000149011612,1,0


summary,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,48889,49042,48729,48868,48889,48889,48885,48732,48889,48886,48887,48732,38858,48885,48731
mean,1.9016769263678946E7,1.02037532075E8,6.749591589946438E7,NaN,9.8700230823495E7,40.007362675159236,40.36302551530165,-73.94211531636516,153.4116304375,152.22910444708097,7.120441017039295,23.257613067388984,1.383749806789856,7.654597524803109,112.61170507479838
stddev,1.098279572074488E7,8.709090084371348E7,7.855358174017501E7,NaN,1.395831475564171E8,9.163872354104724,6.467103792263658,1.4037923691806642,512.1846329459124,238.54053086016683,20.80626648438555,44.55781343932858,2.504724514613329,34.823823045226966,131.6121077449069
min,2539,1 Bed Apt in Utopic Williamsburg,2438,#NAME?,197400421,-74.08151,-74.16254,-74.24442,-73.90783,0,0,0,0.0,0,0
25%,9471997,2.4544724E7,7788268,475.0,40.64699,40.68771,40.68982,-73.98311,59.0,69,1,1,0.19,1,0
50%,19677579,7.1886811E7,30671432,5.1024536E7,40.64699,40.74557,40.7228,-73.95572,86.0,105,3,5,0.72,1,44
75%,29152037,1.74786681E8,107434423,NaN,1.97400421E8,40.78304,40.76299,-73.9364,145.0,175,5,23,2.01,2,226
max,36487245,"ﾏﾝﾊｯﾀﾝ､駅から徒歩4分でどこに行くのにも便利な場所!女性の方希望,ｷﾚｲなお部屋｡",274321313,현선,Woodside,Woodside,40.91306,190.0,Shared room,10000,1250,629,365.0,365,365


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49073 entries, 0 to 49072
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48889 non-null  float64
 1   name                            49042 non-null  object 
 2   host_id                         48729 non-null  float64
 3   host_name                       48868 non-null  object 
 4   neighbourhood_group             48889 non-null  object 
 5   neighbourhood                   48889 non-null  object 
 6   latitude                        48885 non-null  float32
 7   longitude                       48732 non-null  float32
 8   room_type                       48889 non-null  object 
 9   price                           48886 non-null  float64
 10  minimum_nights                  48887 non-null  float64
 11  number_of_reviews               48732 non-null  float64
 12  last_review                     

(49073, 16)

In [0]:
#Dropping all null rows in dataset
temp_df = df.dropna()
display(temp_df.head(5))
temp_df.toPandas().isnull().sum()

#Keeping only required columns, by dropping name, latitude, longitude, calculated_host_listings_count
temp_df = temp_df.drop("name", "host_name", "calculated_host_listings_count")
display(temp_df.head(5))

#Cleaned dataset
cleaned_df = temp_df


id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.647491455078125,-73.97236633300781,Private room,149,1,9,2018-10-19,0.20999999344348907,6,365
2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362014770508,-73.98377227783203,Entire home/apt,225,1,45,2019-05-21,0.3799999952316284,2,355
3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68513870239258,-73.95976257324219,Entire home/apt,89,1,270,2019-07-05,4.639999866485596,1,194
5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851150512695,-73.9439926147461,Entire home/apt,80,10,9,2018-11-19,0.10000000149011612,1,0
5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.7476692199707,-73.9749984741211,Entire home/apt,200,3,74,2019-06-22,0.5899999737739563,1,129


id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,availability_365
2539,2787,Brooklyn,Kensington,40.647491455078125,-73.97236633300781,Private room,149,1,9,2018-10-19,0.20999999344348907,365
2595,2845,Manhattan,Midtown,40.75362014770508,-73.98377227783203,Entire home/apt,225,1,45,2019-05-21,0.3799999952316284,355
3831,4869,Brooklyn,Clinton Hill,40.68513870239258,-73.95976257324219,Entire home/apt,89,1,270,2019-07-05,4.639999866485596,194
5022,7192,Manhattan,East Harlem,40.79851150512695,-73.9439926147461,Entire home/apt,80,10,9,2018-11-19,0.10000000149011612,0
5099,7322,Manhattan,Murray Hill,40.7476692199707,-73.9749984741211,Entire home/apt,200,3,74,2019-06-22,0.5899999737739563,129


In [0]:
cleaned_df.write.format("delta").mode("overwrite").partitionBy("neighbourhood_group").save("abfss://silver@adls12s.dfs.core.windows.net/airbnb_cleaned_data")